# Diseño de la solución


Instrucciones para ejecutar en Visual Studio Code

#### Preparación del entorno:

* Asegúrate de tener Python instalado (versión 3.6 o superior)

* Instala los paquetes necesarios ejecutando en la terminal:

* pip install numpy pandas matplotlib scikit-learn

#### Configuración del archivo:

* Crea un nuevo archivo llamado pinguins_knn.py en VS Code

* Copia el código anterior

* Asegúrate de que el archivo pinguins.csv esté en la misma carpeta

#### Ejecución:

* Abre la terminal integrada en VS Code (Ctrl+`)

#### Ejecuta el script con:

* python pinguins_knn.py

* Características principales

#### Orientación a objetos:

* Clase base abstracta KNNBase

* Implementación concreta KNNClassifier

* Clase Penguin para encapsular los datos

#### Funcionalidad completa:

* Método fit() para entrenamiento

* Método predict() para clasificación

* Cálculo de distancias euclidianas

* Soporte para diferentes valores de k

#### Visualización:

* Gráfico 2D con reducción PCA

* Destacado de vecinos más cercanos

* Leyenda interactiva

#### Manejo de datos:

* Carga desde CSV

* Normalización de características

* División entrenamiento/prueba

#### Sobrecarga de operadores:

* __eq__ para comparar pingüinos

* __add__ para combinar clasificadores


# Implementación mínima exigida


In [1]:
import numpy as np
from abc import ABC, abstractmethod
from collections import Counter

class PenguinSample:
    """Clase para representar ejemplares de pingüinos"""
    
    def __init__(self, features, species=None):
        """
        Inicializa un pingüino con sus características
        
        Args:
            features (np.array): Array con las medidas [bill_length, bill_depth, flipper_length, body_mass]
            species (str, optional): Especie del pingüino. Defaults to None.
        """
        self._features = np.array(features)
        self._species = species
    
    @property
    def features(self):
        return self._features
    
    @features.setter
    def features(self, value):
        self._features = np.array(value)
    
    @property
    def species(self):
        return self._species
    
    @species.setter
    def species(self, value):
        self._species = value
    
    def __eq__(self, other):
        """Dos pingüinos son iguales si tienen exactamente las mismas medidas"""
        return np.array_equal(self.features, other.features)
    
    def __add__(self, other):
        """Combina dos pingüinos en una lista (para simular conjunto de entrenamiento)"""
        return [self, other]
    
    def __repr__(self):
        return f"PenguinSample(features={self.features}, species={self.species})"


class KNNBase(ABC):
    """Clase abstracta base para el clasificador KNN"""
    
    @abstractmethod
    def fit(self, X, y):
        """Almacena los datos de entrenamiento"""
        pass
    
    @abstractmethod
    def distance(self, p1, p2):
        """Calcula la distancia entre dos puntos"""
        pass
    
    @abstractmethod
    def predict(self, X_new, k=3):
        """Predice la clase para nuevos datos"""
        pass


class KNNClassifier(KNNBase):
    """Implementación concreta del clasificador KNN para pingüinos"""
    
    def __init__(self):
        self.X_train = None
        self.y_train = None
        self.classes = None
    
    def fit(self, X, y):
        """
        Almacena las observaciones de entrenamiento
        
        Args:
            X (np.array): Features de entrenamiento
            y (np.array): Etiquetas de entrenamiento
        """
        self.X_train = np.array(X)
        self.y_train = np.array(y)
        self.classes = np.unique(y) if y.dtype == object else None
    
    def distance(self, p1, p2):
        """
        Calcula la distancia Euclidiana entre dos puntos
        
        Args:
            p1 (np.array): Primer punto
            p2 (np.array): Segundo punto
            
        Returns:
            float: Distancia Euclidiana
        """
        return np.sqrt(np.sum((p1 - p2)**2))
    
    def predict(self, X_new, k=3):
        """
        Predice la especie para nuevos datos usando KNN
        
        Args:
            X_new (np.array): Nuevos datos a predecir
            k (int, optional): Número de vecinos. Defaults to 3.
            
        Returns:
            list: Predicciones para cada punto en X_new
        """
        if self.X_train is None or self.y_train is None:
            raise ValueError("El modelo no ha sido entrenado. Llame a fit() primero.")
            
        # Asegurarse que X_new es 2D
        X_new = np.atleast_2d(X_new)
        predictions = []
        
        for x in X_new:
            # Calcula distancias a todos los puntos de entrenamiento
            distances = [self.distance(x, x_train) for x_train in self.X_train]
            
            # Obtiene los índices de los k vecinos más cercanos
            k_indices = np.argsort(distances)[:k]
            
            # Obtiene las etiquetas de los k vecinos más cercanos
            k_nearest_labels = [self.y_train[i] for i in k_indices]
            
            # Votación mayoritaria
            most_common = Counter(k_nearest_labels).most_common(1)
            predictions.append(most_common[0][0])
        
        return predictions[0] if len(predictions) == 1 else predictions


# Ejemplo de uso con datos de pingüinos
if __name__ == "__main__":
    # Datos de ejemplo (simulando el dataset penguins)
    # En la práctica, aquí cargarías tus datos reales como en el ejemplo Iris
    X_train = np.array([
        [39.1, 18.7, 181, 3750],
        [39.5, 17.4, 186, 3800],
        [40.3, 18.0, 195, 3250],
        [36.7, 19.3, 193, 3450],
        [39.3, 20.6, 190, 3650],
        [38.9, 17.8, 181, 3625],
        [39.2, 19.6, 195, 4675],
        [41.1, 17.6, 182, 3200],
        [38.6, 21.2, 191, 3800],
        [34.6, 21.1, 198, 4400]
    ])
    
    y_train = np.array([
        'Adelie', 'Adelie', 'Chinstrap', 'Adelie', 
        'Chinstrap', 'Adelie', 'Gentoo', 'Adelie', 
        'Chinstrap', 'Gentoo'
    ])
    
    # Estandarización (como en el ejemplo Iris)
    mu = X_train.mean(axis=0)
    sigma = X_train.std(axis=0)
    X_train = (X_train - mu) / sigma
    
    # Nuevos datos para predecir (sin estandarizar)
    X_new = np.array([
        [38.5, 18.5, 185, 3700],
        [40.5, 19.0, 190, 4000],
        [37.0, 20.0, 192, 3500]
    ])
    
    # Estandarizar los nuevos datos con los mismos parámetros
    X_new = (X_new - mu) / sigma
    
    # Crear y entrenar el modelo
    knn = KNNClassifier()
    knn.fit(X_train, y_train)
    
    # Probar con diferentes valores de k
    print("Clases disponibles:", knn.classes)
    for k in [1, 3, 5]:
        predictions = knn.predict(X_new, k=k)
        print(f"\nPredicciones con k={k}:")
        for i, pred in enumerate(predictions):
            print(f"Pingüino {i+1}: {pred}")
    
    # Ejemplo con un solo pingüino
    single_penguin = np.array([38.0, 18.0, 180, 3600])
    single_penguin = (single_penguin - mu) / sigma
    prediction = knn.predict(single_penguin, k=3)
    print(f"\nPredicción para pingüino individual: {prediction}")

Clases disponibles: None

Predicciones con k=1:
Pingüino 1: Adelie
Pingüino 2: Adelie
Pingüino 3: Adelie

Predicciones con k=3:
Pingüino 1: Adelie
Pingüino 2: Adelie
Pingüino 3: Chinstrap

Predicciones con k=5:
Pingüino 1: Adelie
Pingüino 2: Adelie
Pingüino 3: Chinstrap

Predicción para pingüino individual: Adelie
